In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm 
import math
from random import sample

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [61]:
df_train = pd.read_csv('./../../../Assignment_2/Data/part_4_train_tt.csv')
df_test = pd.read_csv('./../../../Assignment_2/Data/part_4_test_tt.csv')

In [62]:
df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

In [63]:
df_train.head(10)

,User_1,User_19,User_11,User_17,User_8,User_3,User_38,User_113,User_100,User_28,User_27,User_43,User_15,User_16,User_89,User_112,User_4,User_71,User_108,User_37,User_85,User_7,User_13,User_20,User_52,User_2,User_14,User_31,User_6,User_95,User_109,User_70,User_10,User_21,User_114,User_102,User_87,User_47,User_5,User_44,User_33,User_34,User_35,User_29,User_26,User_74,User_42,User_49,User_99,User_22,User_116,User_92,User_68,User_30,User_41,User_75,User_24,User_12,User_117,User_9,User_110,User_119,User_101,User_36,User_115,User_18,User_144,User_118,User_120,User_25,User_23,User_86,User_90,User_73,User_142,User_141,User_76,User_32,User_50,User_125,User_145,User_97,User_61,User_39,User_148,User_127,User_128,User_46,User_121,User_103,User_122,User_60,User_83,User_40,User_53,User_48,User_146,User_107,User_51,User_123,User_143,User_45,User_54,User_63,User_124,User_138,User_93,User_64,User_126,User_58,User_57,User_106,User_79,User_62,User_78,User_133,User_82,User_55,User_66,User_139,User_56,User_91,User_111,User_59,User_132,User_130,User_67,User_104,User_131,User_65,User_77,User_84,User_80,User_96,User_136,User_129,User_69,User_134,A_Create Application complete,A_Submitted complete,W_Handle leads schedule,W_Handle leads withdraw,W_Complete application schedule,A_Concept complete,W_Complete application start,A_Accepted complete,O_Create Offer complete,O_Created complete,O_Sent (mail and online) complete,W_Complete application complete,W_Call after offers schedule,W_Call after offers start,A_Complete complete,W_Call after offers suspend,W_Call after offers resume,A_Cancelled complete,O_Cancelled complete,W_Call after offers ate_abort,W_Complete application suspend,W_Complete application resume,W_Complete application ate_abort,W_Validate application schedule,W_Validate application start,A_Validating complete,O_Returned complete,W_Validate application suspend,W_Validate application ate_abort,W_Call incomplete files schedule,W_Call incomplete files start,A_Incomplete complete,W_Call incomplete files suspend,W_Call incomplete files resume,W_Call incomplete files ate_abort,O_Sent (online only) complete,W_Handle leads start,W_Handle leads complete,W_Validate application resume,W_Validate application complete,W_Call incomplete files complete,W_Complete application withdraw,W_Call after offers complete,W_Handle leads suspend,W_Handle leads resume,W_Assess potential fraud schedule,W_Assess potential fraud start,W_Assess potential fraud suspend,W_Assess potential fraud resume,W_Assess potential fraud complete,W_Handle leads ate_abort,W_Shortened completion schedule,W_Shortened completion start,W_Shortened completion suspend,W_Shortened completion resume,O_Accepted complete,A_Pending complete,W_Personal Loan collection schedule,W_Personal Loan collection start,W_Personal Loan collection suspend,W_Personal Loan collection resume,W_Assess potential fraud ate_abort,W_Assess potential fraud withdraw,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,case:LoanGoal,case:ApplicationType,case:RequestedAmount,first_timestamp,trace_duration,next_event
0,6.0,9.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1.0,1.0,1.0,1,1,1.0,1,1,1,1.0,1.0,1,1,1,0,0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,35000.0,120.0,True,366.08,False,0.0,35000.0,Home improvement,New credit,35000.0,2016-01-01 13:00:04.360000+00:00,81831.911,False
1,6.0,0.0,0.0,11.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [64]:
columns = list(df_train.columns)

last_ones = columns[201:]

In [65]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255 entries, 0 to 3254
Columns: 214 entries, User_1 to next_event
dtypes: bool(3), float64(131), int64(77), object(3)
memory usage: 5.3+ MB


In [67]:
def feature_engineering(df_train):
    df_train['first_timestamp'] = pd.to_datetime(df_train['first_timestamp'])

    # day
    df_train['Day'] = df_train['first_timestamp'].dt.day
    # month
    df_train['Month'] = df_train['first_timestamp'].dt.month
    # hour
    df_train['Start_hour'] = df_train['first_timestamp'].dt.hour
    # minute
    df_train['Start_minute'] = df_train['first_timestamp'].dt.minute
    # second
    df_train['Start_second'] = df_train['first_timestamp'].dt.second
    # Monday is 0 and Sunday is 6
    df_train['Start_weekday'] = df_train['first_timestamp'].dt.weekday
    # week of the year
    df_train['Start_week_of_year'] = df_train['first_timestamp'].dt.week

    df_train.drop('first_timestamp', inplace=True, axis=1)

    return df_train

In [68]:
df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)

/var/folders/nd/m2w3gccj0hx7zslrqlw6v9z80000gn/T/ipykernel_26986/1275838501.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_train['Start_week_of_year'] = df_train['first_timestamp'].dt.week


In [69]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255 entries, 0 to 3254
Columns: 220 entries, User_1 to Start_week_of_year
dtypes: bool(3), float64(131), int64(84), object(2)
memory usage: 5.4+ MB


In [59]:
col_dict = {}
dtypes_list = dict(df_train.dtypes)
for pair in dtypes_list.items():
    key = str(pair[1])
    col_dict[key] = col_dict.get(key, []) + [pair[0]]

In [70]:
col_dict['object']

['case:LoanGoal', 'case:ApplicationType']

In [107]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB


In [108]:
y_train = df_train['next_event']
x_train = df_train.drop('next_event', inplace=False, axis=1)

y_test = df_test['next_event']
x_test = df_test.drop('next_event', inplace=False, axis=1)

In [109]:
# 2 sub-pipelines, one for numeric features, other for categorical ones
numeric_pipe = make_pipeline(StandardScaler())
categorical_pipe = make_pipeline(OneHotEncoder())

# Using categorical pipe for features A,B,C, numeric pipe otherwise
preprocessor = make_column_transformer((categorical_pipe, col_dict['object']), remainder='passthrough')

# Combine with learning algorithm in another pipeline

pipe = make_pipeline(preprocessor, MultinomialNB())

In [110]:
result = pipe.fit(x_train, y_train)

In [111]:
result

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['case:LoanGoal',
                                                   'case:ApplicationType'])])),
                ('multinomialnb', MultinomialNB())])

In [112]:
pipe.score(x_test, y_test)

0.6645649432534678